In [16]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
import pickle 

import torch
import cv2
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
from torchvision.transforms import ToTensor
from sklearn.preprocessing import Normalizer
from PIL import Image
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


In [17]:
data=np.load("100-Bollywood_celebrity-faces-embeddings.npz")

In [18]:
emdTrainX, trainy, emdTestX, test=data["arr_0"],data["arr_1"],data["arr_2"],data["arr_3"]

In [19]:
img_embeddings=np.concatenate((emdTrainX,emdTestX),axis=0)
# scaler = StandardScaler()
# img_embeddings=scaler.fit_transform(img_embeddings)

In [20]:
print(img_embeddings.shape)

(11803, 512)


In [21]:
np.random.shuffle(img_embeddings)# to shuffle the image embeddings
d={"im_id":list(range(img_embeddings.shape[0]))}

In [22]:
data=pd.DataFrame(d)

In [23]:
data["emb"]=img_embeddings.tolist()

In [25]:
data["emb"]=data["emb"].apply(lambda x: np.array(x))

## Clustering

In [33]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from kneed import KneeLocator
from sklearn.metrics import silhouette_score
import hdbscan

In [34]:
X=img_embeddings
dbscan = hdbscan.HDBSCAN(min_cluster_size=12)
dbscan.fit(X)
dbscan_predictions = dbscan.labels_
data['Cluster_DBSCAN_OPT'] = dbscan_predictions

In [35]:
np.unique(dbscan_predictions)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
       84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98])

In [36]:
a=list(dbscan_predictions)
a.count(-1)

4876

In [37]:
#Silhoutte_score without outliers
silhouettescore = silhouette_score(np.stack(data["emb"][dbscan_predictions!=-1]), dbscan_predictions[dbscan_predictions!=-1])
print(silhouettescore)

0.2643709064450349


In [38]:
X_train = data[data['Cluster_DBSCAN_OPT'] != -1][["emb"]].reset_index(drop=True)
y_train = data[data['Cluster_DBSCAN_OPT'] != -1][['Cluster_DBSCAN_OPT']].reset_index(drop=True)
X_test = data[data['Cluster_DBSCAN_OPT'] == -1][["emb"]].reset_index(drop=True)

In [39]:
KNC = KNeighborsClassifier(n_neighbors=5)
KNC.fit(np.stack(X_train["emb"].values), y_train.values.ravel())
KNC_predictions = KNC.predict(np.stack(X_test["emb"].values))

In [40]:
data['Cluster_Hybrid'] = data['Cluster_DBSCAN_OPT']
data.loc[data['Cluster_DBSCAN_OPT'] == -1, 'Cluster_Hybrid'] = KNC_predictions

In [41]:
silhouettescore = silhouette_score(np.stack(data["emb"]), dbscan_predictions)

In [42]:
silhouettescore

0.03417451681261255